In [1]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.integrate import solve_ivp

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 100

### Code up single-state IVP problem

This might not be perfect since this will likely ignore our battery.